In [3]:
import pandas as pd
import numpy as np

In [5]:
df= pd.read_csv("Summaries/Final_Df.csv")

In [6]:
df.head()

,Unnamed: 0,CustomerID,Gender,Location,Tenure_Months,Transaction_ID,Transaction_Date,Product_SKU,Product_Description,Product_Category,Quantity,Avg_Price,Delivery_Charges,Coupon_Status,Month,Coupon_Code,Discount_pct,GST,Invoice_Value
0,0,12346,F,New York,31,38223,2019-09-15,GGOEAAAJ080816,Android Men's Engineer Short Sleeve Tee Charcoal,Apparel,1,19.99,75.00,Used,Sep,SALE30,0.3,0.18,91.51174
1,1,12346,F,New York,31,38223,2019-09-15,GGOEGOAR013099,Google Stylus Pen w/ LED Light,Office,2,5.50,75.00,Used,Sep,OFF30,0.3,0.10,83.47000
2,2,12347,M,New York,20,23979,2019-03-24,GGOEGOAQ020099,Four Color Retractable Pen,Office,1,2.99,14.41,Clicked,Mar,OFF30,0.3,0.10,17.69900
3,3,12347,M,New York,20,23979,2019-03-24,GGOEGOCT019199,Red Spiral Google Notebook,Office,1,9.99,14.41,Used,Mar,OFF30,0.3,0.10,22.10230
4,4,12347,M,New York,20,23980,2019-03-24,GGOENEBJ079499,Nest Learning Thermostat 3rd Gen-USA - Stainle...,Nest-USA,1,149.00,6.50,Used,Mar,ELEC30,0.3,0.10,121.23000


In [7]:
df["Transaction_Date"] = pd.to_datetime(df.Transaction_Date)

In [8]:
df["Month_Num"] = pd.to_datetime(df.Transaction_Date).dt.month

In [9]:
df.groupby("CustomerID")["Transaction_Date"].max()-df.groupby("CustomerID")["Transaction_Date"].min()

CustomerID
12346     0 days
12347   223 days
12348   119 days
12350     0 days
12356     0 days
          ...   
18259     0 days
18260   105 days
18269    76 days
18277     0 days
18283    73 days
Name: Transaction_Date, Length: 1468, dtype: timedelta64[ns]

In [10]:
data_final = df.groupby("CustomerID").agg({"Invoice_Value":np.mean,"Delivery_Charges":np.mean,"Quantity":np.mean,"Location":max,"Gender":max,"Coupon_Status":max,"Tenure_Months":max})

In [11]:
data_final["Days_Bw_Purchase"] = df.groupby("CustomerID")["Transaction_Date"].max()-df.groupby("CustomerID")["Transaction_Date"].min()

In [12]:
data_final["Quantity"] = data_final.Quantity.round()

In [13]:
data_final.head()

,Invoice_Value,Delivery_Charges,Quantity,Location,Gender,Coupon_Status,Tenure_Months,Days_Bw_Purchase
CustomerID,,,,,,,,
12346,87.490870,75.000000,2.0,New York,F,Used,31,0 days
12347,261.447399,11.085667,6.0,New York,M,Used,20,223 days
12348,73.458954,8.571739,9.0,California,M,Used,39,119 days
12350,86.319722,7.522353,1.0,California,M,Used,25,0 days
12356,55.761917,17.708056,2.0,Chicago,F,Used,31,0 days


In [14]:
data_final["Days_Bw_Purchase"] = data_final.Days_Bw_Purchase.dt.days

In [15]:
def group(x):
    if x<31:
        return "0-30 Days"
    elif x>30 and x<61:
        return "30-60 Days"
    elif x>60 and x<91:
        return "60-90 Days"
    else:
        return "90+ Days"

In [16]:
data_final["Purchase_Category"] = data_final.Days_Bw_Purchase.apply(group)

In [17]:
data_final = pd.get_dummies(data_final,columns = ["Location","Gender","Coupon_Status"],drop_first=True)

In [18]:
data_final.head()

,Invoice_Value,Delivery_Charges,Quantity,Tenure_Months,Days_Bw_Purchase,Purchase_Category,Location_Chicago,Location_New Jersey,Location_New York,Location_Washington DC,Gender_M,Coupon_Status_Not Used,Coupon_Status_Used
CustomerID,,,,,,,,,,,,,
12346,87.490870,75.000000,2.0,31,0,0-30 Days,0,0,1,0,0,0,1
12347,261.447399,11.085667,6.0,20,223,90+ Days,0,0,1,0,1,0,1
12348,73.458954,8.571739,9.0,39,119,90+ Days,0,0,0,0,1,0,1
12350,86.319722,7.522353,1.0,25,0,0-30 Days,0,0,0,0,1,0,1
12356,55.761917,17.708056,2.0,31,0,0-30 Days,1,0,0,0,0,0,1


In [19]:
data_final.columns

Index(['Invoice_Value', 'Delivery_Charges', 'Quantity', 'Tenure_Months',
       'Days_Bw_Purchase', 'Purchase_Category', 'Location_Chicago',
       'Location_New Jersey', 'Location_New York', 'Location_Washington DC',
       'Gender_M', 'Coupon_Status_Not Used', 'Coupon_Status_Used'],
      dtype='object')

In [20]:
X = data_final.drop(["Purchase_Category","Days_Bw_Purchase"],axis=1)

In [21]:
Y = data_final.Purchase_Category

### Model

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_test, Y_train,Y_test = train_test_split(X,Y,test_size=0.2,random_state=12345)

In [24]:
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape

((1174, 11), (294, 11), (1174,), (294,))

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [26]:
log_reg = LogisticRegression(C=0.01, solver='liblinear').fit(X_train,Y_train)
yhat = log_reg.predict(X_test)
print("Train set Accuracy: ", metrics.accuracy_score(Y_train, log_reg.predict(X_train)))
print("Test set Accuracy: ", metrics.accuracy_score(Y_test, yhat))   

Train set Accuracy:  0.626916524701874
Test set Accuracy:  0.6462585034013606


In [27]:
print(metrics.classification_report(Y_test, yhat))  

              precision    recall  f1-score   support

   0-30 Days       0.65      1.00      0.79       190
  30-60 Days       0.00      0.00      0.00        11
  60-90 Days       0.00      0.00      0.00        10
    90+ Days       0.00      0.00      0.00        83

    accuracy                           0.65       294
   macro avg       0.16      0.25      0.20       294
weighted avg       0.42      0.65      0.51       294



C:\Users\pc\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
from sklearn.ensemble import GradientBoostingClassifier

In [29]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,max_depth=1, random_state=0).fit(X_train, Y_train) 
yhat = clf.predict(X_test)

In [31]:
print(metrics.classification_report(Y_test, yhat))  

              precision    recall  f1-score   support

   0-30 Days       0.70      0.84      0.76       190
  30-60 Days       0.00      0.00      0.00        11
  60-90 Days       0.00      0.00      0.00        10
    90+ Days       0.47      0.35      0.40        83

    accuracy                           0.64       294
   macro avg       0.29      0.30      0.29       294
weighted avg       0.58      0.64      0.61       294



In [32]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

In [33]:
dtc = DecisionTreeClassifier()

params = {"criterion":np.array(["mse","gini"]),
                              "splitter":np.array(["best","random"]),
                              "max_depth":np.arange(1,13,1),
         "min_samples_split":np.arange(2,16,1)}

grid_tree = GridSearchCV(dtc,params,cv=5)

In [34]:
grid_tree.fit(X_train,Y_train)

C:\Users\pc\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\model_selection\_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\pc\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\model_selection\_validation.py", line 531, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\pc\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\tree\_classes.py", line 890, in fit
    super().fit(
  File "C:\Users\pc\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\tree\_classes.py", line 333, in fit
    criterion = CRITERIA_CLF[self.criterion](self.n_outputs_,
KeyError: 'mse'

  warnings.warn("Estimator fit failed. The score on this train-test"
C:\Users\pc\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\model_selection\_validation.py:548: F

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': array(['mse', 'gini'], dtype='<U4'),
                         'max_depth': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
                         'min_samples_split': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15]),
                         'splitter': array(['best', 'random'], dtype='<U6')})

In [35]:
estimator = grid_tree.best_estimator_

In [36]:

grid_tree.best_score_

0.6286015639207129

In [37]:
print(metrics.classification_report(Y_test, estimator.predict(X_test)))  

              precision    recall  f1-score   support

   0-30 Days       0.65      0.97      0.77       190
  30-60 Days       0.00      0.00      0.00        11
  60-90 Days       0.00      0.00      0.00        10
    90+ Days       0.33      0.04      0.07        83

    accuracy                           0.64       294
   macro avg       0.24      0.25      0.21       294
weighted avg       0.51      0.64      0.52       294



C:\Users\pc\AppData\Local\Programs\PythonCodingPack\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
